In [67]:
# scraping Q&As of Li Fei from Haodf.com

import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import re

# weburl = "http://www.haodf.com/wenda/dflifei_g_4642049195.htm" # a test page
webheader = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) \
        AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36'}
urls = []
data = []

def getSinglePage(pageurl):
    req = urllib.request.Request(url=pageurl, headers=webheader)
    try:
        webPage = urllib.request.urlopen(req)
        """
        print(type(webPage))
        print(webPage.geturl())
        print(webPage.info())
        print(webPage.getcode())
        """
    except:
        print('\033[33m Warning:\033[0m Fail to open:', pageurl)
        pass # failed +1
        return None
    else:
        bsObj = BeautifulSoup(webPage, "lxml")
        # questions = bsObj.findAll("div", {"class":"h_s_cons_info"})
        # answers = bsObj.findAll("div", {"class":"h_s_cons_docs"})
        patient = bsObj.find(lambda tag: tag.get_text() == '病情描述：')
        if patient:
            symptom = ''.join(list(map(str, patient.next_siblings))).replace('<br/>', '')
            # get rid of bingli attachment
            symptom = re.sub(r'<[^>]*>[^<>]*</[^>]*>|\n|\r', '', symptom)
            patient = patient.parent.parent.parent.parent.parent # any better solution?
            symptom_time = patient.find("div", {"class":"yh_l_times"}).get_text()
            for reply in patient.next_siblings: # diagnosis follows symptom
                try:
                    diagnosis = reply.find("div", {"class":"h_s_cons_docs"})
                    diagnosis = diagnosis.find("h3").get_text()
                except:
                    continue # not a reply by doctor, keep searching...
                else:
                    diagnosis_time = reply.find("div", {"class":"yh_l_times"}).get_text()
                    break # only the first reply is valid
            """
            print(symptom)
            print(symptom_time)
            print(diagnosis)
            print(diagnosis_time)
            """
            data.append({'url':pageurl, 'symptom':symptom, 'symptom_time':symptom_time, \
                        'diagnosis':diagnosis, 'diagnosis_time':diagnosis_time})
        else: # not Q&As of desired pattern (see README.md)
            pass # dirty data +1

# getSinglePage(weburl)

for i in range(1,2):
    listurl = ''.join(['http://dflifei.haodf.com/zixun/list.htm?type=&p=',str(i)])
    req = urllib.request.Request(url=listurl, headers=webheader)
    listPage = urllib.request.urlopen(req)
    print('Now scraping page', i)
    bsObj = BeautifulSoup(listPage, "lxml")
    listtable = bsObj.find("div", {"class":"zixun_list"})
    for post in listtable.findAll("a", {"class":"td_link"}):
        urls.append(post['href'])
        getSinglePage(post['href'])

print('Scraped', len(data), 'valid data out of', len(urls), 'posts.')

Now scraping page 1
Scraped 9 valid data out of 25 posts.


In [68]:
data

[{'diagnosis': '可以去听课，也可以带孩子来我看看',
  'diagnosis_time': '2016-08-26 ',
  'symptom': '目前情况：1，不和同龄的孩子玩。主要没有兴趣，有时看见大孩子跑闹也想参与，但不知怎么去做。在大人的提示下可以和同龄孩子打招呼，握手，拥抱。。2，能够和大人快乐的玩耍，有要求，有疑问会向大人提问。吐字清楚。如：妈妈，爷爷做什么去了？但不会和大人进行一些比较社会的交流。比如你今天去海洋馆看见什么了？为什么哭了？但是会回答：有没有看见海龟等问题。偶尔也能比较吃力的描述一段刚刚发生的事，语言组织能力不灵活，句子大部分都是引用大人，或电视里听过的。基本都在场景中。3，精细动作差，拿笔划线无力，只会画圈。拿勺子吃饭僵硬不灵活，不会使用工具，除非特意去教。大运动，机械记忆较好，会骑儿童三轮车。6个月就能指认一些字。会唱儿歌，能够跟着老师跳一些简单舞蹈。自己不会单独跳。喜欢看认知型的动画片，比如天线宝宝，巧虎，米奇妙妙屋。爱反复看同一集。不爱看社会行为型的动画片：如粉红猪小妹，我是悠悠。不爱听故事。可能听不懂。偶尔有一两个听下去了。就反复听那一个。4行为刻板，会在某一段时间，坚持某个行为轨迹，改变会大哭大闹。喜欢风扇，转轮子。5，情感对大人依赖，没有安全感，一岁之前没发现这些，除了睡眠不好，很少哭闹，爱玩乐，有时还会逗陌生人玩，对母亲很亲，经常爱笑，会主动亲母亲，看见母亲下班会高兴跑过来。母乳依赖强，每天晚上要吃母乳6～7次，基本吃两口就睡了。哪怕摔破头，只要给予母乳安慰就不哭了。一周岁时，母亲开始值夜班，强制性断奶，母亲离家一周。大宝哭了近两个星期。情绪反常近2个月，直到开始愿意吃奶瓶才有好转，但是性情没有之前平稳，爱急躁，哭闹，不能等待。2岁半之前，二宝出生前还能够接受。那时经常更换抚养人，但是父母下班都是带着玩的，二宝出生时当时父母离家半月在外地住院待产。带小宝回来一个月后。大宝开始情绪失常，经常哭闹，很难哄好，夜间哭闹，每夜闹好几次。明显焦虑，抑郁。后来母亲给予多关注，让父亲陪他睡后，约半年后逐渐好转。那时依赖妈妈，和小弟弟抢妈妈，妈妈上班会哭闹，要求和妈妈一起去，看见妈妈下班会飞奔扑上去。5个月前妈妈外出两个月后，就不要妈妈了，(因中间又开始夜间哭闹被爸爸多次训斥，就跑去和爷爷睡了)，目前依赖爷爷，让妈妈走